In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%connect_info

In [ ]:
import seaborn as sns 
from ubergauss.optimization import df_remove_duplicates as clean
from ubergauss.optimization import get_best, pareto_scores
import numpy as np
import ubergauss.tools as ut


In [ ]:
#df = ut.loadfile("/home/ubuntu/repos/cellsaw/application/res_scib1")
# application/optimization.py will run the optimization
import pandas as pd
df = pd.read_csv("/home/ubuntu/repos/cellsaw/application/out.csv")
id = df.columns.tolist()
df.fillna(0, inplace=True)

In [ ]:
df.corr()


In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
# sc22 = sc2.pivot(index="method", columns="dataset", values="batch_mean")
# geomean = np.sqrt(df.label_mean * df.batch_mean)
# df['geomean'] = geomean



In [ ]:
#ee = 'label_mean	label_std	batch_std batch_mean	time'.split()
# df =df.drop(labels = ee, axis =1)


In [ ]:
cols = [col for col in df.columns.tolist() if col != 'geomean']
cols

In [ ]:
df.sort_values(by='score', ascending=False)


In [ ]:
import seaborn as sns
# sns.pairplot(df, hue='geomean')




import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

'''
outlier_threshold
k
hub1/2 algo k
'''
sns.lineplot(df, y='score', x= 'outlier_threshold') 
plt.show()
sns.lineplot(df, y='score', x= 'k') 
plt.show()
sns.lineplot(df, y='score', x= 'hub1_k', hue = 'hub1_algo') 
plt.show()
sns.lineplot(df, y='score', x= 'hub2_k', hue = 'hub2_algo') 

plt.show()
sns.boxplot(df, y='score', x= 'hub2_algo') 
plt.show()
sns.boxplot(df, y='score', x= 'hub1_algo') 



In [ ]:
df.sort_values(by='geomean', ascending=False)

In [ ]:
sns.lineplot(df, y= 'geomean', x = 'hub1_algo', hue='hub2_algo') 
# k -> 10 .. 15 11 seems good
# outlier threshold 0.76

df.corr()

In [ ]:
id.remove('time')
id.remove('batch_std')
id.remove('label_std')
id.remove('label_mean')
id.remove('batch_mean')
# the idea is to put everyting into a score columns so we can just do geomean on tha one column 
df = df.melt(id_vars = id ,  value_vars=['batch_mean','label_mean'], var_name='target', value_name='score')




In [ ]:
import scalp

In [ ]:
from scipy.stats import gmean
# next we group by stuff by parameter combo

grp = ['dim'] # this is added by the optimizer
for line in scalp.mkgraphParameters.split('\n'):
    if line:
        grp.append(line.split(' ')[0])


grouped = df.groupby(grp)

if False: # debug
    print (grp)
    pd.set_option('display.expand_frame_repr', False)
    for name in grouped.groups.keys():
        print(grouped.get_group(name))






In [ ]:
# for each group we want the keys and the geomean
data =[ list(key) + [gmean(group.score)] for key, group in grouped]

In [ ]:
# then we build a dataframe from this 
columns = grp + ['score']
df3=pd.DataFrame(data, columns=columns)
df3=df3.sort_values(by='score')
pd.set_option('display.max_rows', None)
df3

In [ ]:
z= df3.loc[786].to_dict()
z.pop('dim')
z.pop('score')
z

In [ ]:
g = sns.PairGrid(df3, y_vars=['score'], x_vars=columns, height=4, aspect=1.5)
g.map(sns.scatterplot)

In [ ]:
df3[ (df3.score > .5)  &  (df3.copy_lsa_neighbors == 0) & (df3.intra_neighbors_mutual == 1)]

In [ ]:
sns.pairplot(df3[ (df3.score > .5)  &  (df3.copy_lsa_neighbors == 0) & (df3.intra_neighbors_mutual == 1)], hue='score')

# HERE WE STOP

In [ ]:
id = df.columns.tolist()
pscores = pareto_scores(df, data=None,method = id)

In [ ]:
import pandas as pd
newdf = [dict(zip(id,v)) for v,score in pscores]
for d,s in zip(newdf, pscores):
    d['score'] = s[1]
newdf = pd.DataFrame(newdf)
g=sns.pairplot(newdf, hue = 'score', height=1.5,  palette = 'magma')
    

In [ ]:
import pandas as pd
newdf = [dict(zip(id,v)) for v,score in pscores]
for d,s in zip(newdf, pscores):
    d['score'] = -s[1]
newdf = pd.DataFrame(newdf)
get_best(newdf)

In [ ]:
from ubergauss import optimization as uo
newdf.sort_values(by=['score'])[-20:].corr()

In [ ]:
newdf.sort_values(by=['score'])[-20:]

In [ ]:
df = ut.loadfile("/home/ubuntu/repos/cellsaw/application/res_scib1")
df['score'] = [ ch+bat+sil for ch,sil,bat in zip(df.class_cohesion, df.silhouette, df.batch_cohesion)]
df_clean = clean(df)
# sns.pairplot(df_clean, hue = 'score', height=1.5, corner=True)
g=sns.pairplot(df_clean, hue = 'silhouette', height=1.5,  palette = 'magma')

def overwrite(x, y, label=None, color=None, cmap=None, **kwargs):
    plt.scatter(x,y)

g.map_lower(overwrite)

df_clean['FAIL'] = [ np.isnan(z)  for z in df.score]
get_best(df)
get_best(df,column = 'silhouette')
df_clean.corr(method='spearman')

In [ ]:
df

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", module="anndata")
warnings.filterwarnings("ignore", message=".*Attempted to fork from a non-main thread.*")


In [ ]:
df = ut.loadfile("/home/ubuntu/repos/cellsaw/application/res_TS1")
df['score'] = [ ch+bat+sil for ch,sil,bat in zip(df.class_cohesion, df.silhouette, df.batch_cohesion)]
df_clean = clean(df)
sns.pairplot(df_clean, hue = 'score', height=1.5, corner=True)
df['FAIL'] = [ np.isnan(z)  for z in df.score]
get_best(df)
df_clean.corr(method='spearman')

In [ ]:
import pandas

In [ ]:
import optimization as opt
from ubergauss.optimization import nutype 
data =opt.getdata()
d2 = [[d] for d in data]

In [ ]:

o = nutype.nutype(opt.space, opt.ho_eval,d2, numsample=16 )

In [ ]:
%%time
o.opti()
o.print()
o.opti()
o.print()
o.opti()
o.print()

In [ ]:
 {'hub1_algo': 1.0, 'hub1_k': 13.021210834947537, 'hub2_algo': 3.0, 'hub2_k': 11.274756377019376, 'k': 17.235099709198174, 'outlier_threshold': 0.6046280880758059, 'datafield': 0.0, 'time': 253.87748432159424, 'score': 0.4716044284270298}